In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [2]:
df = pd.read_csv('./data/gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [3]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
X.drop(labels = ['id'], axis = 1, inplace = True)

In [4]:
# Segregation into numerical and categorical columns
num_columns = X.columns[X.dtypes != 'object']
cat_columns = X.columns[X.dtypes == 'object']

In [5]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [10]:
num_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler())
    ]

)

preprocess=ColumnTransformer([
('num_pipeline',num_pipeline,num_columns),
('cat_pipeline',cat_pipeline,cat_columns)
])

In [11]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=10)

In [12]:
X_train = pd.DataFrame(preprocess.fit_transform(X_train), columns=preprocess.get_feature_names_out())
X_test = pd.DataFrame(preprocess.transform(X_test), columns=preprocess.get_feature_names_out())

In [13]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.780167,0.719352,-0.637086,-0.843944,-0.808252,-0.760667,-1.139986,-1.550963,0.682694
1,1.793874,-1.307964,0.403474,1.681406,1.751577,1.530872,-0.134981,0.297649,0.017348
2,-0.758536,-1.584416,0.403474,-0.690619,-0.762865,-0.833184,-0.134981,-1.550963,0.682694
3,0.063426,-0.478607,-0.637086,0.301483,0.281037,0.254572,0.870024,0.297649,-1.313344
4,-0.563861,0.627201,-0.637086,-0.483179,-0.508697,-0.441592,0.870024,0.297649,2.013386
